In [22]:
import pandas as pd 

itemset = pd.read_csv('data\itemset.csv')
itemset


,name,brand,description,product_id,availability,price (NRs),avg_rating
0,Purina and reg; Kit and amp; Kaboodle and reg...,Kit 'N Kaboodle,Purina and reg; Kit and amp; Kaboodle and reg...,5139253,1,1423.8125,3.743000
1,Chicken Soup for the Soul All Life Stages Dry ...,Chicken Soup for the Soul,Chicken Soup for the Soul All Life Stages Dry ...,5329365,1,2136.3125,4.265498
2,ZuPreem and reg; NutBlend Bird Food,ZuPreem,ZuPreem and reg; NutBlend Bird Food,5167132,1,2136.3125,4.272700
3,Top Paw and reg; Teal Floral Dog Leash: 6-ft l...,Top Paw,Top Paw and reg; Teal Floral Dog Leash: 6-ft l...,5307432,1,2017.5625,3.000000
4,Dentley's and reg; Rawhide-Free Mini Chomping ...,Dentley's,Dentley's and reg; Rawhide-Free Mini Chomping ...,5291357,1,1898.8125,4.736800
...,...,...,...,...,...,...,...
204,Greenies and amp;trade; Feline Greenies SmartB...,Greenies,Greenies and amp;trade; Feline Greenies SmartB...,5286358,1,2255.0625,4.803600
205,Vitakraft and reg; Menu Canary and amp; Finch...,Vitakraft,Vitakraft and reg; Menu Canary and amp; Finch...,5166955,1,592.5625,5.000000
206,Blue Buffalo and reg; Homestyle Recipe Senior...,Blue Buffalo,Blue Buffalo and reg; Homestyle Recipe Senior...,5200131,1,343.1875,4.590200
207,Petlinks and reg; Marble Mayhem and amp;trade;...,Petlinks,Petlinks and reg; Marble Mayhem and amp;trade;...,5305962,1,830.0625,3.529400


In [23]:
import re 

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)


In [24]:
itemset['name'] = itemset["name"].apply(clean_title)
itemset['brand'] = itemset["brand"].apply(clean_title)
itemset['description'] = itemset["description"].apply(clean_title)
itemset


,name,brand,description,product_id,availability,price (NRs),avg_rating
0,Purina and reg Kit and amp Kaboodle and reg O...,Kit N Kaboodle,Purina and reg Kit and amp Kaboodle and reg O...,5139253,1,1423.8125,3.743000
1,Chicken Soup for the Soul All Life Stages Dry ...,Chicken Soup for the Soul,Chicken Soup for the Soul All Life Stages Dry ...,5329365,1,2136.3125,4.265498
2,ZuPreem and reg NutBlend Bird Food,ZuPreem,ZuPreem and reg NutBlend Bird Food,5167132,1,2136.3125,4.272700
3,Top Paw and reg Teal Floral Dog Leash 6ft long...,Top Paw,Top Paw and reg Teal Floral Dog Leash 6ft long...,5307432,1,2017.5625,3.000000
4,Dentleys and reg RawhideFree Mini Chomping Che...,Dentleys,Dentleys and reg RawhideFree Mini Chomping Che...,5291357,1,1898.8125,4.736800
...,...,...,...,...,...,...,...
204,Greenies and amptrade Feline Greenies SmartBit...,Greenies,Greenies and amptrade Feline Greenies SmartBit...,5286358,1,2255.0625,4.803600
205,Vitakraft and reg Menu Canary and amp Finch Food,Vitakraft,Vitakraft and reg Menu Canary and amp Finch Food,5166955,1,592.5625,5.000000
206,Blue Buffalo and reg Homestyle Recipe Senior ...,Blue Buffalo,Blue Buffalo and reg Homestyle Recipe Senior ...,5200131,1,343.1875,4.590200
207,Petlinks and reg Marble Mayhem and amptrade Ro...,Petlinks,Petlinks and reg Marble Mayhem and amptrade Ro...,5305962,1,830.0625,3.529400


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
vectorizer = TfidfVectorizer(ngram_range=(1,2)) # ngram_range takes two words into accounts for searching
tfidf = vectorizer.fit_transform(itemset['name']) #trun values into a matrix basically apply the Tfidfvectorizer

tfidf


<209x1886 sparse matrix of type '<class 'numpy.float64'>'
	with 4268 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(name):
    query_vec = vectorizer.transform([name]) 
    cossimilarity = cosine_similarity(query_vec, tfidf).flatten() 
    #identifies the indices of the top 5 items that have the highest cosine similarity with the query item
    indices = np.argpartition(cossimilarity, -5)[-10:] #sort the movies with ascending cosine similarity [-5] selects the last 5 movies as they have highest cosine similarity
    results = itemset.iloc[indices][::-1]
    results = results[['name', 'price (NRs)']]
    return results


In [27]:
import ipywidgets as widgets
from IPython.display import display

item_input = widgets.Text(
    value='dog food',
    placeholder='Search Item',
    description='Item:',
    disabled=False
)
item_list = widgets.Output()

def on_type(data):
    with item_list:
        item_list.clear_output()
        name = data["new"]
        if len(name) > 2:
            display(search(name))

item_input.observe(on_type, names='value')


display(item_input, item_list)


Text(value='dog food', description='Item:', placeholder='Search Item')

Output()